# Ingeniería Mecatrónica - Robótica Industrial

### David Rozo Osorio, I.M, M.Sc.

## Práctica No. 2 - Microcontroladores

- Objetivo: diseñar e implementar código sobre el microcontrolador Raspberry Pi Pico


- Procedimiento:
  1. IDE - VSCode.
  1. Configuración inicial y especificaciones técnicas.
  1. Primer programa - *Blink*.
  1. Entradas y Salidas Digitales.
  1. Entradas y Salidas Análogas.
  1. Multihilos.
  1. Interrupciones.
  1. Ejecución.

### IDE - VSCode


![vscode](./graph/vscode.png)

Página web oficial: https://code.visualstudio.com/download

- Instalar las siguientes extensiones:
  - CPython Extension Pack
  - Cython
  - micro:bit
  - MicroPython
  - Pico-W-Go


### Configuración inicial

Para que el microcontrolador Raspberry Pi Pico funcione con **Micropython** se debe cargar el *firmware* de MicroPython más reciente en el *board* [(ver Pag. Web fabricante)](https://micropython.org/download/rp2-pico/rp2-pico-latest.uf2). Este procedimiento se realiza por medio del siguiente procedimiento:

1. Descargar el *firmware*.
1. Iniciar el microcontrolador en modo *Boot*.
   1. Conectar el cable USB-A en el computador.
   1. Presionar el botón (*BOOTSEL*) de la placa.
   1. Conectar el cable micro-USB a la tarjeta, sosteniendo el botón presionado.
   1. El PC debe reconocer la tarjeta como si fuera una memoria USB.
1. Copiar el *firmware* en la carpeta del dispositivo.
1. La tarjeta se debe reiniciar y el PC debe reconocerla como un dispositivo conectado a un puerto serial (COM#)

### Microcontrolador Pinout 


![02_Pinout_Full.png](./graph/02_Pinout_Full.png)

### Programa Blink

El presente código permite generar una operación con el **LED** interno que se encuentra en la placa.

```python
# libreria machine interaccion hardware
import machine
# libreria para el manejo de tiempo
import utime

# instacia objeto led, Clase Pin --> keyword: Pin.OUT
led = machine.Pin(25, machine.Pin.OUT) # LED conectado a este puerto 25-(built-in)

# ciclo infinito de ejecucion
while True:
    # accion sobre el objeto equivalente a asigancion
    led.on()
    # delay en segundos
    utime.sleep(1)
    led.off()
    utime.sleep(1)
```
El programa debe ser guardado con el nombre de `main.py` para que el *bootloader* lo ejecute cuando se energice el microcontrolador.


### Programa de Entradas y Salidas Digitales

El presente código permite utilizar una entrada y una salida digital de tal forma que se pueda comprender su funcionamiento.

<img src="./graph/io_digital_pushbutton.png" width=400 />

```python
# libreria machine interaccion hardware
import machine
# libreria para el manejo de tiempo
import utime

# instacia objeto led, Clase Pin --> keyword: Pin.OUT
led = machine.Pin(25, machine.Pin.OUT)
# instancia de objeto boton, como Entrada, adicioalmente se activa con resistencia Pull_UP
boton = machine.Pin(15, machine.Pin.IN, machine.Pin.PULL_UP)

# ciclo infinito de ejecucion
while True:
    # condicional if-else
    if boton.value() == 1:
        # delay en milisegundos 
        utime.sleep_ms(200)
        # accion sobre el objeto equivalente a asigancion
        led.on()
        print("on")
    else:
        led.value(0)
        print("off")
        utime.sleep_ms(200)

```
El programa debe ser guardado con el nombre de `main.py` para que el *bootloader* lo ejecute cuando se energice el microcontrolador.

### Programa uso de los Timers

El presente código permite desarrollar operaciones con los *timers* internos que tiene el hardware

```python
# libreria machine interaccion hardware
import machine
# libreria para el manejo de tiempo
import utime

# funcion que llama el timer
def ticks(timer):
    global led
    led.toggle()
    print("timer loop")

# instacia objeto led, Clase Pin --> keyword: Pin.OUT
led = machine.Pin(25, machine.Pin.OUT)

timer = machine.Timer()
timer.init(freq = 2.5, mode = machine.Timer.PERIODIC, callback=ticks)

while True:
    print("main loop")
    utime.sleep_ms(100)

```
El programa debe ser guardado con el nombre de `main.py` para que el *bootloader* lo ejecute cuando se energice el microcontrolador.

### Programa para variables ADC

El presente código permite desarrollar operaciones con los pines de conversión Análogo/Digital. En particular el sensor de temperatura que se encuentra conectado al GPIO-4.

**Nota.** El convertidor ADC es de 12 bits, sin embargo, MicroPython internamente lo convierte a 16 bits.

```python
# libreria machine interaccion hardware
import machine
# libreria para el manejo de tiempo
import utime

sensor_temp = machine.ADC(4) # al pin 4 esta conectado el sensor de temperatura
conversion_factor = 3.3/ (65535)

while True:
    reading = sensor_temp.read_u16()*conversion_factor
    temperature = 27 - (reading - 0.706)/0.001721

    print("temp: ",temperature)
    utime.sleep_ms(100)
```
El programa debe ser guardado con el nombre de `main.py` para que el *bootloader* lo ejecute cuando se energice el microcontrolador.

### Programa para variables ADC y Señal PWM

El presente código permite desarrollar operaciones con los pines de conversión Análogo/Digital. Adicionalmente, se definirá una señal PWM para controlar la luminosidad de un LED.

**Nota.** El convertidor ADC es de 12 bits, sin embargo, MicroPython internamente lo convierte a 16 bits.

<img src="./graph/ADC_Pot.png" width=400 />

```python
# libreria machine interaccion hardware
import machine
# libreria para el manejo de tiempo
import utime

potenciometro = machine.ADC(26) # al pin 26 esta conectado un potenciometro
frecuencia = 5000

led = machine.PWM(machine.Pin(25)) # LED conectado a este puerto 25-(built-in)
led.freq(frecuencia)


conversion_factor = 3.3/ (65535)

while True:
    reading = potenciometro.read_u16()
    print(reading*conversion_factor)
    led.duty_u16(reading)
    utime.sleep(0.5)
```
El programa debe ser guardado con el nombre de `main.py` para que el *bootloader* lo ejecute cuando se energice el microcontrolador.

### Programa Multihilo 

El presente código permite desarrollar operaciones con los 2 núcleos del chip RP2040 

<img src="./graph/multihilo_led.png" width=400 />

```python
# libreria machine interaccion hardware
import machine
# libreria para el manejo de tiempo
import utime
# libreria multihilos
import _thread


def nucleo_2():
    while True:
        utime.sleep_ms(50)
        led1.toggle()


led1 = machine.Pin(14, machine.Pin.OUT) # LED conectado a este puerto
led2 = machine.Pin(15, machine.Pin.OUT)

_thread.start_new_thread(nucleo_2,())

while True:
    utime.sleep_ms(1500)
    led2.toggle()
```
El programa debe ser guardado con el nombre de `main.py` para que el *bootloader* lo ejecute cuando se energice el microcontrolador.

### Programa con IRQ

El presente código permite desarrollar operaciones con Interrupciones

<img src="./graph/iqr_encoder.png" width=400 />

```python
# libreria machine interaccion hardware
import machine
# libreria para el manejo de tiempo
import utime

# funcion que llama el timer
def _motorEncoderCallbackL(pin):
    global motor_l_encoder_cnt, motor_l_encoder2
    if motor_l_encoder2.value() > 0:
        motor_l_encoder_cnt += 1
    else:
        motor_l_encoder_cnt -= 1


motor_l_encoder_cnt = 0
motor_encoder_cnt_total_l = 0

motor_l_encoder1 = machine.Pin(13, machine.Pin.IN, machine.Pin.PULL_UP)
motor_l_encoder2 = machine.Pin(12, machine.Pin.IN, machine.Pin.PULL_UP)

motor_l_encoder1.irq(trigger=machine.Pin.IRQ_FALLING, handler=_motorEncoderCallbackL) # IRQ_RISING

while True:
    print("Encoder pulses: ", motor_l_encoder_cnt)
    utime.sleep_ms(100)
```
El programa debe ser guardado con el nombre de `main.py` para que el *bootloader* lo ejecute cuando se energice el microcontrolador.

## Material de Referencia

- [Raspberry Pi Pico Python SDK](https://datasheets.raspberrypi.com/pico/raspberry-pi-pico-python-sdk.pdf)
- [Get Started with MicroPython on Raspberry Pi Pico](https://hackspace.raspberrypi.com/downloads/eyJfcmFpbHMiOnsibWVzc2FnZSI6IkJBaHBBaThSIiwiZXhwIjpudWxsLCJwdXIiOiJibG9iX2lkIn19--d43ee613629bddf78bc41c1479c2acb2ec6ef34e/RPi_PiPico_Digital_v10.pdf)
- [MicroPython API](http://docs.micropython.org/en/latest/rp2/quickref.html)
